<img src="https://risk-engineering.org/static/img/logo-RE.png" width="100" alt="" style="float:right;margin:15px;">

This notebook is an element of the [risk-engineering.org courseware](https://risk-engineering.org/). It can be distributed under the terms of the [Creative Commons Attribution-ShareAlike licence](https://creativecommons.org/licenses/by-sa/4.0/).

Author: Eric Marsden <eric.marsden@risk-engineering.org>

---

In this notebook, we illustrate features of the NumPy, Pandas, Scikit-Learn and statsmodels libraries features for building **linear regression models** in Python. Consult the [accompanying course materials](https://risk-engineering.org/linear-regression-analysis/) for background on linear regression analysis, some notes on when the technique is useful, and to download this content as a Jupyter/Python notebook.

# Linear regression analysis of combined cycle power plant data

In [ ]:
import numpy
import pandas
import matplotlib.pyplot as plt
import scipy.stats
from sklearn.linear_model import LinearRegression # Regression Model
from sklearn.model_selection import train_test_split # to split train and test sets
plt.style.use("bmh")
%config InlineBackend.figure_formats=["png"]

We will analyze data from a combined cycle power plant to attempt to build a predictive model for output power. The data comes from the UCI machine learning repository.

http://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant

The dataset contains 9568 data points collected from a combined cycle power plant over 6 years, when power plant was under full load. A combined cycle power plant is composed of gas turbines, steam turbines and heat recovery steam generators. Electricity is generated by gas & steam turbines, which are combined in one cycle. Three ambient variables affect the performance of the gas turbine, and exhaust vacuum affects the performance of the steam turbine. 

Data consists of hourly averages taken from various sensors located around the plant that record the ambient variables every second.

Let’s load it into Python and examine it using the `pandas` library. For convenience, we have unzipped the dataset and made it web accessible.

In [ ]:
data = pandas.read_csv("https://risk-engineering.org/static/data/CCPP.csv")
print("dataset type is:", type(data), "length:", len(data), "shape:", data.shape)
dataset=data.copy()

We have 4 features (AT, V, AP, RH) and one label (PE)

Let's take a look at the data content

In [ ]:
data.head()

The table below provides the meaning of the various columns.

| Meaning | Name | Range |
| --- | --- | --- |
| Ambient Temperature | AT | 1.81 – 37.11°C
| Ambient Pressure | AP |  992.89 – 1033.30 millibar
| Relative Humidity | RH | 25.56% – 100.16%
| Exhaust Vacuum  | V |  25.36 – 81.56 cm Hg
| Net hourly electrical energy output | PE | 420.26 – 495.76 MW |


In [ ]:
data.describe()

In [ ]:
data.info()

There is no missing data. In this case we don't need to do any imputation.

We can check the 'null' values in this way as well.

In [ ]:
data.isnull().values.any()

Let's get column names

In [ ]:
print(data.columns.values)

## Visualization

We can obtain a first impression of the dependency between variables by examining a multidimensional scatterplot. (See https://pandas.pydata.org/docs/reference/api/pandas.plotting.scatter_matrix.html to have more details about the 'scatter_matrix' syntax)

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(data, diagonal="kde");

In this matrix, the diagonal contains a plot of the distribution of each variable. We observe that:

- there is an approximately linear relationship between AT and the V

- there is an approximately weak linear relationship between V and negative of AP

In [ ]:
data.plot(kind="scatter", x='AT', y='V', grid=True)

In [ ]:
data.plot(kind="scatter", x='V', y='AP', grid=True)

We can also generate a 3D plot of the observations, which can sometimes help to interpret the data more easily. Here we plot PE as a function of AT and V.

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(projection="3d")
fig.add_axes(ax)
ax.scatter(data["AT"], data["V"], data["PE"])
ax.set_xlabel("AT")
ax.set_ylabel("V")
ax.set_zlabel("PE")
ax.set_facecolor("white")

#matplotlib inline
data.hist(bins=50, figsize=(20,15))
plt.show()

## Looking for correlation

You can easily compute the standard correlation coefficient (also called Pearson's r) between every pair of attributes using the 'corr()' method.

In [ ]:
corr_matrix= data.corr()

In [ ]:
corr_matrix["AT"].sort_values(ascending=False)

As expected there is a strong positive correlation (0.8) between "AT" and "V".

In [ ]:
corr_matrix["V"].sort_values(ascending=False)

Theres is a weak negative correlation (-0.4) between "V" and "AP"

## Getting the 'labels' and 'features'

In [ ]:
labels=data.pop("PE")

In [ ]:
print("label shape:", labels.shape, "and type:", type(labels))

In [ ]:
print("features shape:", data.shape, "and type:", type(data))

In [ ]:
labels.info()

In [ ]:
X_features=data.columns.values
print(X_features)

## Split training and test set

In [ ]:
X = data.values  # values converts it into a numpy array
Y = labels.values 

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

## Model Training

We will created a fitted linear model using the formula API of the `scikit-learn` library. We use all the observations except for PE as predictor variables in the multiple linear regression.

In [1]:
linear_model = LinearRegression()
#X must be a 2-D Matrix
linear_model.fit(X_train, Y_train) 

NameError: name 'LinearRegression' is not defined

### View Parameters 
The $\mathbf{w}$ and $\mathbf{b}$ parameters are referred to as 'coefficients' and 'intercept' in scikit-learn. In other term hte model function can be written as $f_{w,b}(\vec{x})$

In [ ]:
b = linear_model.intercept_
w = linear_model.coef_
print(f"w = {w:}, b = {b:0.2f}")
print(f"'manual' prediction: f_wb = wx+b : {1200*w + b}")

This means that the best formula to estimate output power as a function of AT, V, AP and RH is

$$PE = − 1.97309513   AT − 0.2382388   V + 0.06009385  AP − 0.15660587   RH + 456.67$$

For any particular observation (values for the predictor variables), we can use the linear model to estimate the output variable PE.

In [ ]:
linear_model.predict(pandas.DataFrame({"AT": [9.48], "V": [44.71], "AP": [1019.12], "RH": [66.43]}))

The predicted output power for this combination of inputs is 478 MW (note that we report the result with the same number of significant figures as that in our inputs).

In [ ]:
some_data=X_test[:5,:]
some_labels=Y_test[:5]

In [ ]:
some_data.shape

In [ ]:
some_labels_predicted = linear_model.predict(some_data)

In [ ]:
some_labels_predicted.shape

In [ ]:
print("Predictions:", some_labels_predicted)

In [ ]:
print("Labels:", some_labels)

Let’s measure this regression model’s RMSE on the whole train‐
ing set using Scikit-Learn’s mean_squared_error function

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
lin_mse = mean_squared_error(some_labels, some_labels_predicted)
lin_rmse = numpy.sqrt(lin_mse)
lin_rmse

The Root Mean Squared Error is good enough. Good catch

## Residuals plots

We check the residuals of each predictor variable for any pattern that might indicate that a linear model is not appropriate for this dataset.

In [ ]:
residuals = linear_model.predict(X_test) - Y_test
plt.plot(X_test[:,0], residuals, ".", alpha=0.5)
plt.xlabel("AT")
plt.ylabel("Residual");

In [ ]:
plt.plot(X_test[:,1], residuals, ".", alpha=0.5)
plt.xlabel("V")
plt.ylabel("Residual");

In [ ]:
plt.plot(X_test[:,2], residuals, ".", alpha=0.5)
plt.xlabel("AP")
plt.ylabel("Residual");

In [ ]:
plt.plot(X_test[:,3], residuals, ".", alpha=0.5)
plt.xlabel("RH")
plt.ylabel("Residual");

Indeed, except for a minor quadratic shape to the residuals of variable AT (which we will ignore here), the residuals look random, without any systematic feature apparent that might indicate that our linear model is not appropriate for this data. 

We also check that the variance of the residuals is normally distributed by plotting a histogram or a QQ plot of the residuals, as shown below.

In [ ]:
plt.hist(residuals, bins=40, alpha=0.5)
plt.title("Histogram of the residuals", weight="bold");

In [ ]:
scipy.stats.probplot(residuals, dist=scipy.stats.norm, plot=plt.figure().add_subplot(111));

## Calculate accuracy

You can calculate this accuracy of this model by calling the `score` function.

In [ ]:
print("Accuracy on training set:", linear_model.score(X_train, Y_train))

In [ ]:
print("Accuracy on test set:", linear_model.score(X_test, Y_test))

# Gradient Descent
Scikit-learn has a gradient descent regression model [sklearn.linear_model.SGDRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html#examples-using-sklearn-linear-model-sgdregressor).  Like your previous implementation of gradient descent, this model performs best with normalized inputs. [sklearn.preprocessing.StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler) will perform z-score normalization as in a previous lab. Here it is referred to as 'standard score'.

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler

### Scale/normalize the training data

In [ ]:
scaler = StandardScaler()
X_norm = scaler.fit_transform(X_train)
print(f"Peak to Peak range by column in Raw        X:{numpy.ptp(X_train,axis=0)}")   
print(f"Peak to Peak range by column in Normalized X:{numpy.ptp(X_norm,axis=0)}")

scaler = StandardScaler()
X_norm = scaler.fit_transform(X_train)
print(f"Peak to Peak range by column in Raw        X:{np.ptp(X_train,axis=0)}")   
print(f"Peak to Peak range by column in Normalized X:{np.ptp(X_norm,axis=0)}")

### Create and fit the regression model

In [ ]:
sgdr = SGDRegressor(max_iter=1000)
sgdr.fit(X_norm, Y_train)
print(sgdr)
print(f"number of iterations completed: {sgdr.n_iter_}, number of weight updates: {sgdr.t_}")

### View parameters
Note, the parameters are associated with the *normalized* input data. The fit parameters are very close to those found in the previous lab with this data.

In [ ]:
b_norm = sgdr.intercept_
w_norm = sgdr.coef_
print(f"model parameters:                   w: {w_norm}, b:{b_norm}")

### Make predictions
Predict the targets of the training data. Use both the `predict` routine and compute using $w$ and $b$.

In [ ]:
# make a prediction using sgdr.predict()
y_pred_sgd = sgdr.predict(X_norm)
# make a prediction using w,b. 
y_pred = numpy.dot(X_norm, w_norm) + b_norm  
print(f"prediction using np.dot() and sgdr.predict match: {(y_pred == y_pred_sgd).all()}")

print(f"Prediction on training set:\n{y_pred[:4]}" )
print(f"Target values \n{Y_train[:4]}")

### Plot Results
Let's plot the predictions versus the target values.

In [ ]:
# plot predictions and targets vs original features 
dlc = dict(dlblue = '#0096ff', dlorange = '#FF9300', dldarkred='#C00000', dlmagenta='#FF40FF', dlpurple='#7030A0')
fig,ax=plt.subplots(1,4,figsize=(12,3),sharey=True)
for i in range(len(ax)):
    ax[i].scatter(X_train[:,i],Y_train, label = 'target')
    ax[i].set_xlabel(X_features[i])
    ax[i].scatter(X_train[:,i],y_pred,color=dlc["dlorange"], label = 'predict')
ax[0].set_ylabel("PE"); ax[0].legend();
fig.suptitle("target versus prediction using z-score normalized model")
plt.show()

## Goodness of fit

In [ ]:
import statsmodels.formula.api as smf

#fit linear regression model
model = smf.ols(formula="PE ~ AT + V + AP + RH", data=dataset).fit()

#view model summary
print(model.summary())

The $R^2$ for this model is 0.929, which means that it explains roughly 93% of the variance of the output power. Interpretation of this number is dependent on the application (for instance, models used to understand health effects tend to have lower $R^2$ values than those used for physical models), but is quite satisfactory for this application. 